In [1]:
import soundfile as sf
import sounddevice as sd
import numpy as np
import sys
from queue import Queue

In [2]:
sd.default.device

[1, 3]

In [8]:
sd.query_devices()

  0 Microsoft Sound Mapper - Input, MME (2 in, 0 out)
> 1 마이크(Realtek(R) Audio), MME (2 in, 0 out)
  2 Microsoft Sound Mapper - Output, MME (0 in, 2 out)
< 3 Realtek HD Audio 2nd output(Rea, MME (0 in, 2 out)
  4 스피커(Realtek(R) Audio), MME (0 in, 2 out)
  5 Speakers (Realtek HD Audio output), Windows WDM-KS (0 in, 2 out)
  6 마이크 (Realtek HD Audio Mic input), Windows WDM-KS (2 in, 0 out)
  7 Headphones (Realtek HD Audio 2nd output), Windows WDM-KS (0 in, 2 out)
  8 스테레오 믹스 (Realtek HD Audio Stereo input), Windows WDM-KS (2 in, 0 out)
  9 Speakers (Nahimic mirroring Wave Speaker), Windows WDM-KS (0 in, 2 out)

In [4]:
sd.query_devices(1,'input')

{'name': '마이크(Realtek(R) Audio)',
 'hostapi': 0,
 'max_input_channels': 2,
 'max_output_channels': 0,
 'default_low_input_latency': 0.09,
 'default_low_output_latency': 0.09,
 'default_high_input_latency': 0.18,
 'default_high_output_latency': 0.18,
 'default_samplerate': 44100.0}

In [5]:
sd.default.channels

[None, None]

## 녹음

In [6]:
sr = 16000
ch = 1
subtype = 'PCM_16' # sample_width 4byte(32bit)

In [7]:
q = Queue()
def callback(indata, frames, time, status):
    if status:
        print(status, file=sys.stderr)
    # # 소리 낼 때만 녹음
    # if np.max(indata ) > 5e-5:
    q.put(indata.copy())
    
try:
    file = sf.SoundFile('./outputs/sd_microphone_test.wav','w',sr,ch,subtype)
    with file:
        with sd.InputStream(samplerate=sr, channels=ch, callback=callback):
            sig = []
            while True:
                file.write(q.get())
                # sig.extend(q.get())
                # if len(sig) > 16000:
                #     print(len(sig))
                #     sig = []
                # print(np.max(sig))
except KeyboardInterrupt as ke:
    print("Recording finished")
except Exception as e:
    print(e)
    

Recording finished


In [32]:
sig, sr = sf.read('./outputs/sd_microphone_test.wav')
sd.play(sig,sr)
# sd.wait()

## 믹싱 테스트

In [1]:
import librosa

In [5]:
sr = 16000
back_y,_ = librosa.load('./dataset/audio/script1_g_0044-6002-01-01-KSM-F-05-A.wav',sr)
fore_y,_ = librosa.load('./dataset/audio/script1_g_0044-6001-01-01-KSM-F-05-A.wav',sr)


In [6]:
print(f'back shape = {back_y.shape}')
print(f'fore shape = {fore_y.shape}')


back shape = (47040,)
fore shape = (57600,)


In [10]:
mixing = []
for f, b in zip(fore_y,back_y):
    mixing.append(f+b)
mixing = np.array(mixing)
print(mixing.shape)

(47040,)


In [13]:
sd.play(fore_y,sr)
sd.wait()
sd.play(mixing,sr)
sd.wait()

## adding noise with snr

In [67]:
import torch
import math

In [63]:
def load_wav(file):
    y,_ = librosa.load(file, sr)
    return torch.from_numpy(y)

In [64]:
sr = 16000
speech = load_wav('./dataset/audio/script1_g_0044-6002-01-01-KSM-F-05-A.wav')
noise = load_wav('./dataset/audio/script1_g_0044-6001-01-01-KSM-F-05-A.wav')

In [65]:
if speech.shape[0] > noise.shape[0]:
    speech = speech[:noise.shape[0]]
else:
    noise = noise[:speech.shape[0]]

In [66]:
speech_power = speech.norm(p=2)
noise_power = noise.norm(p=2)

In [68]:
snr_dbs = [20,10,3]
noisy_speeches = []
for snr_db in snr_dbs:
    snr = math.exp(snr_db/10)
    scale = snr*noise_power/speech_power
    noisy_speeches.append( (scale*speech + noise)/2 )

In [ ]:
sd.play(noisy_speeches[1],sr)